In [8]:
#!/usr/bin/python
# GoogleMapDownloader.py
# Created by Hayden Eskriett [http://eskriett.com]
#
# A script which when given a longitude, latitude and zoom level downloads a
# high resolution google map
# Find the associated blog post at: http://blog.eskriett.com/2013/07/19/downloading-google-maps/

import urllib.request
from PIL import Image
import os
import math
import csv

class GoogleMapsLayers:
    ROADMAP = "v"
    TERRAIN = "p"
    ALTERED_ROADMAP = "r"
    SATELLITE = "s"
    TERRAIN_ONLY = "t"
    HYBRID = "y"

class GoogleMapDownloader:
    """
    A class which generates high resolution google maps images given
    a latitude, longitude, and zoom level.
    """

    def __init__(self, zoom=12, layer=GoogleMapsLayers.ROADMAP):
        """
        GoogleMapDownloader Constructor
        Args:
            zoom: The zoom level of the location required, ranges from 0 - 23 (default is 12)
            layer: The layer type (default is ROADMAP)
        """
        self._zoom = zoom
        self._layer = layer

    def getXY(self, lat, lng):
        """
        Generates an X,Y tile coordinate based on the latitude, longitude
        and zoom level.
        Args:
            lat: The latitude of the location.
            lng: The longitude of the location.
        Returns:
            An X,Y tile coordinate.
        """
        tile_size = 256
        numTiles = 1 << self._zoom
        point_x = (tile_size / 2 + lng * tile_size / 360.0) * numTiles // tile_size
        sin_y = math.sin(lat * (math.pi / 180.0))
        point_y = ((tile_size / 2) + 0.5 * math.log((1 + sin_y) / (1 - sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size
        return int(point_x), int(point_y)

    def generateImage(self, lat, lng, **kwargs):
        """
        Generates an image by stitching Google Map tiles together.
        Args:
            lat: The latitude of the location.
            lng: The longitude of the location.
            start_x: The top-left x-tile coordinate (default is None).
            start_y: The top-left y-tile coordinate (default is None).
            tile_width: The number of tiles wide the image should be (default is 5).
            tile_height: The number of tiles high the image should be (default is 5).
        Returns:
            A high-resolution Google Map image.
        """
        start_x = kwargs.get('start_x', None)
        start_y = kwargs.get('start_y', None)
        tile_width = kwargs.get('tile_width', 5)
        tile_height = kwargs.get('tile_height', 5)

        if start_x is None or start_y is None:
            center_x, center_y = self.getXY(lat, lng)
            start_x = center_x - tile_width // 2
            start_y = center_y - tile_height // 2

        width, height = 256 * tile_width, 256 * tile_height
        map_img = Image.new('RGB', (width, height))

        for x in range(0, tile_width):
            for y in range(0, tile_height):
                url = f'https://mt0.google.com/vt?lyrs={self._layer}&x=' + str(start_x + x) + '&y=' + str(start_y + y) + '&z=' + str(self._zoom)
                current_tile = str(x) + '-' + str(y)
                urllib.request.urlretrieve(url, current_tile)
                im = Image.open(current_tile)
                map_img.paste(im, (x * 256, y * 256))
                os.remove(current_tile)

        return map_img


def main():
    # Create a new instance of GoogleMapDownloader
    
    proxy_address = '190.181.27.198:6332'  # Change this to your proxy address
    gmd = GoogleMapDownloader(zoom=17, layer=GoogleMapsLayers.SATELLITE)
    
    # Read coordinates from a CSV file
    with open('C:\\Users\\patel1637\\Downloads\\pitWithMB.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header if exists
        
        for _ in range(182):
            next(reader)
        
        for i, row in enumerate(reader):
            lat, lng = map(float, row)
            print(f"Generating image {i+182}...")
            try:
                # Get the high resolution image
                img = gmd.generateImage(lat, lng)
            except IOError:
                print(f"Could not generate the image for coordinates {lat}, {lng}")
                break
            else:
                # Save the image to disk
                img.save(f"MB_Pit_Small/pitImage_{lat}_{lng}.png")
                print(f"Image image_{lat}_{lng} has been successfully generated")

if __name__ == '__main__':
    main()


Generating image 182...
Image image_49.4500185_-98.63495303 has been successfully generated
Generating image 183...
Image image_49.45508_-98.8090276 has been successfully generated
Generating image 184...
Image image_49.45735546_-99.25340335 has been successfully generated
Generating image 185...
Image image_49.45816648_-99.7898888 has been successfully generated
Generating image 186...
Image image_49.45849913_-98.86738675 has been successfully generated
Generating image 187...
Image image_49.45977118_-99.79014554 has been successfully generated
Generating image 188...
Image image_49.46114173_-99.24778628 has been successfully generated
Generating image 189...
Image image_49.46344734_-100.3767656 has been successfully generated
Generating image 190...
Image image_49.46778955_-99.35112212 has been successfully generated
Generating image 191...
Image image_49.49090174_-100.1619287 has been successfully generated
Generating image 192...
Image image_49.50623096_-96.79166514 has been succes

Image image_49.69632661_-95.32384604 has been successfully generated
Generating image 272...
Image image_49.69969531_-95.32081534 has been successfully generated
Generating image 273...
Image image_49.70005544_-95.3162469 has been successfully generated
Generating image 274...
Image image_49.70096628_-96.40867922 has been successfully generated
Generating image 275...
Image image_49.70110662_-96.39947045 has been successfully generated
Generating image 276...
Image image_49.70774104_-99.69215722 has been successfully generated
Generating image 277...
Image image_49.70817585_-95.30764902 has been successfully generated
Generating image 278...
Image image_49.70994005_-95.30266663 has been successfully generated
Generating image 279...
Image image_49.71234945_-96.2409775 has been successfully generated
Generating image 280...
Image image_49.71370307_-96.2545548 has been successfully generated
Generating image 281...
Image image_49.71929609_-95.22015671 has been successfully generated
Gene